# Build a Multi-Layer Neural Network


## Equations Multi-layer
---
$$ Z^{[l]} = W^{[l]}A^{[l-1]} + b^{[l]} $$

$$ A^{[l]} = g^{[l]}(Z^{[l]}) $$

Where $l$ is the `lth` layer.

## Weights Initialization
---
Firstly, weights need to be initialized for different layers. Note that in general, the input is not considered as a layer, but output is.

For `lth` layer, 
- weight $W^{[l]}$ has shape $(n^{[l]}, n^{[l-1]})$
- bias $b^{[l]}$ has shape $(n^{[l]}, 1)$

where $n^{[0]}$ equals the number input feature.

In [2]:
import numpy as np

In [5]:
def weights_init(layers_dim):
    params = {}
    
    n = len(layers_dim)
    for i in range(1, n):
        params['W' + str(i)] = np.random.randn(layers_dim[i], layers_dim[i-1])
        params['b' + str(i)] = np.random.randn(layers_dim[i], 1)
    return params

In [6]:
p = weights_init([2, 5, 1])
p

{'W1': array([[-1.20260404,  1.51803722],
        [ 1.31068456,  3.23918964],
        [ 0.89989452,  0.42286449],
        [ 0.83184057, -0.17971632],
        [-1.13190191, -1.41989977]]),
 'b1': array([[ 0.33066558],
        [ 0.69946101],
        [-0.42476741],
        [ 0.19228531],
        [ 0.61643758]]),
 'W2': array([[ 0.43247981,  1.33887682, -0.54990187,  0.42008628,  0.22424862]]),
 'b2': array([[0.14716179]])}